In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import folium                              #for maps

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
folder='Crime_Boston'
file='crimes_in_boston.csv'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Crime_Boston/crimes_in_boston.csv')# To display the top 5 rows 
df.head(5)

In [ ]:
df.dtypes

A continuación podemos ver que el Lat y Long tienen 19.998 registros menos que Location. 
Shooting es un true unicamente cuando se registro un tiroteo dentro del crimen, a pesar de que sean pocas, es importante considerarlo.

In [ ]:
df.count()

Eliminar duplicados

In [ ]:
df.shape

In [ ]:
duplicate_rows_df = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df.shape)

Esto quiere decir que de las 319073 filas hay 23 duplicadas que eliminaremos.

In [ ]:
df = df.drop_duplicates()
df.shape

In [ ]:
print(df.isnull().sum())

1. Los que no tienen Lat y Long, vemos que es porque se seteo una Location de (0.00000000, 0.00000000). Lo podemos identificar dado que los 19998 que no tenian Lat y Long es igual a la frecuencia de (0.00000000, 0.00000000) en el campo Location.

2. Dada la repeticion del Incident number, siendo "I162030584" el mas repetido, 13 veces, podemos concluir que cada incidente puede tener mas de un registro, indicando distintos OFFENSE_CODE/GROUP/DESCRIPTION. Realizado para agregar detalle a ese incidente.

3. B2 es el distrito con mayor cantidad de registros, sin embargo, al repetirse el Incident Number, tendriamos que eliminar los repetidos para saber cual fue el número exacto de incidentes.

In [ ]:
summary = df.describe(include='all').transpose()
summary

1. Eliminamos la columna OFFENSE_CODE: Es un codigo interno de la policia de Boston, que son agrupados en un mismo OFFENSE_CODE_GROUP, que a nuestro analisis lo enriquece mucho mas, dado que brinda detalle de cuál es el tipo de Incidente. Podríamos buscar analizar los distintos offense code dentro de un mismo code group, pero como no lo vamos a hacer, la columna solo ensuciaría el análisis.

2. En nuestro análisis, creemos que la OFFENSE_DESCRIPTION, no agrega valor. La información principal a analizar se encuentra en el OFFENSE_GROUP.

3. La columna OCCURRED_ON_DATE es la agrupación de las columnas YEAR,	MONTH y	HOUR. Al separar la fecha en esos 3 campos, se facilita el analisis que buscamos, por lo que eliminamos la columna OCCURRED_ON_DATE.

4. Analizaremos las ubicaciones de cada incidente, para eso, usaremos las columnas Lat y Long, que están agrupadas en la columna Location, por lo que eliminaremos está última, que no suma valor.

In [ ]:
df = df.drop(['OFFENSE_CODE', 'OFFENSE_DESCRIPTION', 'OFFENSE_DESCRIPTION', 'OCCURRED_ON_DATE', 'Location'], axis=1)
df.head(5)

Ahora para poder analizar por barrio, limpiamos los INCIDENT_NUMBER duplicados, generando un nuevo dataset, pero manteniendo el anterior, ya que con el detalle que brindan los repetidos se pueden hacer otros analisis.

In [ ]:
duplicate_rows_df = df[df.INCIDENT_NUMBER.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df.shape)

In [ ]:
df_wo_dup_incident = df.drop_duplicates(subset=["INCIDENT_NUMBER"], keep='first')

Eliminamos los duplicados y el "dataset without duplicate incidents" es de 282517 filas y 17 columnas, a diferencia del df que es de 319073 filas y 17 columnas.

In [ ]:
df_wo_dup_incident.shape

Ahora, podemos ver que B2 sigue siendo el DISTRICT con mayores incidentes, con un número de 43403. Podemos ver que al eliminar INCIDENT_NUMBER duplicados, este número bajo desde 49940 incidentes. 

In [ ]:
summary = df_wo_dup_incident.describe(include='all').transpose()
summary

In [ ]:
map

Cantidad de incidentes por mes. Se concluye que Agosto es el mes con mayor cantidad de incidentes y Febrero el de menor cantidad.

In [ ]:
sns.catplot(x='MONTH',
           kind='count',
            height=5, 
            aspect=2,
            palette=sns.color_palette(['blue','green','blue','blue','blue','blue','blue','red','blue','blue','blue','blue']),
           data=df_wo_dup_incident)
plt.xticks(size=20)
plt.yticks(size=20)
plt.xlabel('Mes', fontsize=20)
plt.ylabel('Cantidad de Incidentes', fontsize=20)

Cantidad de incidentes por hora. Se concluye que entre las 16 y las 19hs son los momentos con mayor cantidad de incidentes y entre las 3 y las 6am, los momentos con menor cantidad de incidentes reportados.

In [ ]:
sns.catplot(x='HOUR',
           kind='count',
            height=4,
            aspect=2,
            palette=sns.color_palette(['blue','blue','blue','green','green','green','blue','blue','blue','blue','blue','blue','blue','blue','blue','blue','red','red','red','blue','blue','blue','blue']),
           data=df_wo_dup_incident)
plt.xticks(size=10)
plt.yticks(size=20)
plt.xlabel('Dia', fontsize=20)
plt.ylabel('Cantidad de Incidentes', fontsize=20)

Cantidad de incidentes por dia. Se concluye que no hay grandes variaciones entre los dias, sin embargo, hay una leve cantidad mayor de incidentes durantes los viernes, y menor los domingos, que el resto de los dias.

In [ ]:
sns.catplot(x='DAY_OF_WEEK',
           kind='count',
            height=4,
            aspect=2,
            order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday'],
            palette=sns.color_palette(['blue','blue','blue','blue','red','blue','green']),
           data=df_wo_dup_incident)
plt.xticks(size=10)
plt.yticks(size=20)
plt.xlabel('Dia', fontsize=20)
plt.ylabel('Cantidad de Incidentes', fontsize=20)

Los años con mayor cantidad de reportes son 2017 y 2016 respectivamente, muy por encima de 2018 y 2015, que son los de menores incidentes respectivamente.

In [ ]:
sns.catplot(x='YEAR',
           kind='count',
            height=4,
            aspect=2,
            palette=sns.color_palette(['green','blue','red','blue']),
           data=df_wo_dup_incident)
plt.xticks(size=10)
plt.yticks(size=10)
plt.xlabel('Año', fontsize=20)
plt.ylabel('Cantidad de Incidentes', fontsize=15)

Cantidad de incidentes por tipos de ofensas, divididos por distrito.

In [ ]:
df_per_district = df_wo_dup_incident.groupby(['DISTRICT','OFFENSE_CODE_GROUP'])['OFFENSE_CODE_GROUP'].count()
df_per_district

10 tipos de ofensas mas comunes indicando el distrito y ordenados por cantidad de ofensas de ese tipo en dicho distrito. 

In [ ]:
df_per_district = df_wo_dup_incident.groupby(['DISTRICT','OFFENSE_CODE_GROUP'])['OFFENSE_CODE_GROUP'].agg(['count']).sort_values(by='count', ascending=False).nlargest(10, 'count')
df_per_district

Shootings por distrito, ordenados por cantidad. Vemos que B2, B3 y C11 son los distritos con mas shootings, con mucha diferencia por encima del resto.

In [ ]:
df_shootings = df_wo_dup_incident.groupby(['DISTRICT','SHOOTING'])['SHOOTING'].agg(['count']).sort_values(by='count', ascending=False)
df_shootings

df_shootings['total'] = df_shootings['count'].sum()     
df_shootings['porcentaje'] = df_shootings['count']/df_shootings['total']*100     
df_shootings

In [ ]:
df_shootings.groupby(['DISTRICT']).sum().plot(
    kind='pie', y='count', autopct='%1.0f%%', legend=False, colors = ['blue', 'blue', 'blue','darkred','firebrick','red','blue','blue','blue','blue','blue','blue'])

Calles con mayores incidentes junto con su distrito.

In [ ]:
df_streets = df_wo_dup_incident.groupby(['DISTRICT','STREET'])['STREET'].agg(['count']).sort_values(by='count', ascending=False)
df_streets

Caja y bigotes de shootings por hora, se concluye que la mayor cantidad de tiroteos se encuentran entre las 6 y las 21hs.

In [ ]:
sns.boxplot(data=df_wo_dup_incident, x="HOUR", y="SHOOTING")

Caja y bigotes de shootings por hora en cada uno de los distritos. Se puede observar como en algunos de los distritos, los tiroteos están mas concentrados a la noche, mientras que en otros es durante todo el dia. A1 se distingue al ser el unico que su mayor concentración es por la mañana.

In [ ]:
sns.boxplot(data=df_wo_dup_incident, x="HOUR", y="DISTRICT", hue="SHOOTING", dodge=False).set_title('SHOOTINGS PER HOUR')
plt.legend([],[], frameon=False)

Donut plot de incidentes por distrito

In [ ]:
def show_donut_plot(col):
    
    rating_data = df_wo_dup_incident.groupby(col)[['INCIDENT_NUMBER']].count().head(10)
    plt.figure(figsize = (12, 8))
    plt.pie(rating_data[['INCIDENT_NUMBER']], autopct = '%1.0f%%', startangle = 140, pctdistance = 1.1)

    # create a center circle for more aesthetics to make it better
    gap = plt.Circle((0, 0), 0.5, fc = 'white')
    fig = plt.gcf()
    fig.gca().add_artist(gap)
    
    plt.axis('equal')
    
    cols = []
    for index, row in rating_data.iterrows():
        cols.append(index)
    plt.legend(cols)
    
    plt.title('Donut Plot by ' +str(col), loc='center')
    
    plt.show()

In [ ]:
show_donut_plot('DISTRICT')

Cantidad de ncidentes por tipo de UCR_PART divididos por distrito. Se puede observar que en todos ellos, los de Part Three son los mas repetidos. En algunos barrios (cómo lo es el caso de B3), los de Part One (casos mas graves) parecerian estar mas controlados que los de Part Two. Sin embargo, un caso que sale del molde es el del distrito D4, donde los mas graves (Part One) superan a los medios (Part Two) 

In [ ]:
df_wo_dup_incident2 = df_wo_dup_incident.dropna(subset=['UCR_PART'])
plt.figure(figsize=(8,8))
ax = sns.countplot(x='DISTRICT',data=df_wo_dup_incident2,hue="UCR_PART")
bars = ax.patches
half = int(len(bars)/2)
left_bars = bars[:half]
right_bars = bars[half:]

for left, right in zip(left_bars, right_bars):
    height_l = left.get_height()
    height_r = right.get_height()
    total = height_l + height_r

Por último, armamos un mapa con las longitudes y latitudes de cada incidente, que nos permitirá visualizar y comprender graficamente las ubicaciones en la ciudad.

In [ ]:
location = df_wo_dup_incident.dropna()
location = location[["Lat", "Long", "INCIDENT_NUMBER"]]

In [ ]:
map = folium.Map(location=[location.Lat.mean(), location.Long.mean()], zoom_start=14, control_scale=True)

In [ ]:
for index, location_info in location.iterrows():
    folium.Marker([location_info["Lat"], location_info["Long"]], popup=location_info["INCIDENT_NUMBER"]).add_to(map)

In [ ]:
map